In [1]:
import pandas as pd

In [2]:
rides = pd.read_csv("2011-capitalbikeshare-tripdata.zip")
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226767 entries, 0 to 1226766
Data columns (total 9 columns):
Duration                1226767 non-null int64
Start date              1226767 non-null object
End date                1226767 non-null object
Start station number    1226767 non-null int64
Start station           1226767 non-null object
End station number      1226767 non-null int64
End station             1226767 non-null object
Bike number             1226767 non-null object
Member type             1226767 non-null object
dtypes: int64(3), object(6)
memory usage: 84.2+ MB


In [3]:
rides.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,5th & F St NW,31620,5th & F St NW,W00247,Member
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,Georgia & New Hampshire Ave NW,31104,Adams Mill & Columbia Rd NW,W00357,Member
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,10th & U St NW,31503,Florida Ave & R St NW,W00970,Member
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,Adams Mill & Columbia Rd NW,31106,Calvert & Biltmore St NW,W00346,Casual


In [4]:
del rides["Start station"]
del rides["End station"]

In [5]:
rides['Member type'].value_counts()

Member     979814
Casual     246949
Unknown         4
Name: Member type, dtype: int64

In [6]:
rides = rides[rides['Member type'] != "Unknown"]

In [7]:
rides['Member type'].value_counts()

Member    979814
Casual    246949
Name: Member type, dtype: int64

In [8]:
rides['start'] = pd.to_datetime(rides['Start date'])
rides['start'].iloc[0]

Timestamp('2011-01-01 00:01:29')

In [9]:
rides['start'].tail(10).dt.strftime("%Y-%m-%d")

1226757    2011-12-31
1226758    2011-12-31
1226759    2011-12-31
1226760    2011-12-31
1226761    2011-12-31
1226762    2011-12-31
1226763    2011-12-31
1226764    2011-12-31
1226765    2011-12-31
1226766    2011-12-31
Name: start, dtype: object

In [10]:
rides['date'] = rides['start'].dt.date#strftime("%Y-%m-%d")
rides['hour'] = rides['start'].dt.hour

In [11]:
grouped_as_index = rides.groupby(['date', 'hour', 'Member type']).size()
rides_by_hour = grouped_as_index.to_frame("rides").reset_index()
rides_by_hour.head()

,date,hour,Member type,rides
0,2011-01-01,0,Casual,3
1,2011-01-01,0,Member,13
2,2011-01-01,1,Casual,8
3,2011-01-01,1,Member,30
4,2011-01-01,2,Casual,5


In [12]:
from pandas.tseries.holiday import USFederalHolidayCalendar

In [13]:
import datetime

In [14]:
holidays = pd.DataFrame({
    'date': USFederalHolidayCalendar().holidays(datetime.date(2011,1,1), datetime.date(2015,12,31)).date,
    'is_holiday': True})
holidays.head()

,date,is_holiday
0,2011-01-17,True
1,2011-02-21,True
2,2011-05-30,True
3,2011-07-04,True
4,2011-09-05,True


In [15]:
# too few
rides_by_hour_with_holidays = pd.merge(
    rides_by_hour,
    holidays,
    left_on='date',
    right_on='date')
print(len(rides_by_hour_with_holidays))
rides_by_hour_with_holidays.head()

413


,date,hour,Member type,rides,is_holiday
0,2011-01-17,0,Casual,1,True
1,2011-01-17,0,Member,16,True
2,2011-01-17,1,Casual,1,True
3,2011-01-17,1,Member,15,True
4,2011-01-17,2,Member,8,True


In [16]:
rides_by_hour_with_holidays = pd.merge(
    rides_by_hour,
    holidays,
    left_on='date',
    right_on='date',
    how='left',
    indicator=False)
print(len(rides_by_hour_with_holidays))
rides_by_hour_with_holidays.head()

16417


,date,hour,Member type,rides,is_holiday
0,2011-01-01,0,Casual,3,NaN
1,2011-01-01,0,Member,13,NaN
2,2011-01-01,1,Casual,8,NaN
3,2011-01-01,1,Member,30,NaN
4,2011-01-01,2,Casual,5,NaN


In [17]:
rides_by_hour_with_holidays['is_holiday'] = rides_by_hour_with_holidays['is_holiday'].fillna(False)

In [18]:
rides_by_hour_with_holidays.head()

,date,hour,Member type,rides,is_holiday
0,2011-01-01,0,Casual,3,False
1,2011-01-01,0,Member,13,False
2,2011-01-01,1,Casual,8,False
3,2011-01-01,1,Member,30,False
4,2011-01-01,2,Casual,5,False


https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf
https://www.ncei.noaa.gov/data/global-hourly/doc/CSV_HELP.pdf

There's a "Find a Station" tool, but it's confusing how to use the results. https://www.ncdc.noaa.gov/data-access/land-based-station-data/station-metadata has a link to a [station list file](ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-history.txt). Searching that, it looks like the code for Reagan Airport is 724050 13743. So the file is
https://www.ncei.noaa.gov/data/global-hourly/access/2011/72405013743.csv

In [19]:
weather = pd.read_csv("https://www.ncei.noaa.gov/data/global-hourly/access/2011/72405013743.csv")

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (43,47,51,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
weather['TMP'].str.split(',', n=1, expand=True).head()

,0,1
0,+0056,1
1,+0056,5
2,+0050,5
3,+0039,5
4,+0039,1


In [21]:
weather[['temp_raw', 'temp_validity']] = weather['TMP'].str.split(',', n=1, expand=True)
weather[['temp_raw', 'temp_validity']].head()

,temp_raw,temp_validity
0,+0056,1
1,+0056,5
2,+0050,5
3,+0039,5
4,+0039,1


In [22]:
weather['temp_validity'].value_counts(dropna=False)

5    11266
1     2906
9      378
A        7
6        1
Name: temp_validity, dtype: int64

Meanings:

* 5: Passed QC
* 1: Passed QC
* 9: Outside of limits
* A: suspect but good


In [23]:
weather[weather.temp_validity == "A"].temp_raw

2331    +0220
5071    +0180
6636    +0220
7251    +0330
8191    +0390
8914    +0280
9038    +0250
Name: temp_raw, dtype: object

In [24]:
weather['temp_is_valid'] = weather['temp_validity'].isin(["5", "1", "A"])
weather = weather[weather['temp_is_valid']].copy()

In [25]:
weather['temp_C'] = pd.to_numeric(weather['temp_raw']) / 10

In [26]:
hourly_temp_data = weather.resample('1H', on='date')['temp_C'].mean().reset_index()
hourly_temp_data.head()

KeyError: 'The grouper name date is not found'

In [ ]:
hourly_temp_data['hour'] = hourly_temp_data['date'].dt.hour
hourly_temp_data['date'] = hourly_temp_data['date'].dt.date
hourly_temp_data.head()

In [ ]:
merged_data = pd.merge(
    rides_by_hour_with_holidays,
    hourly_temp_data,
    on=['date', 'hour'])
merged_data.head()

In [ ]:
import seaborn as sns
%matplotlib inline

In [ ]:
sns.jointplot(x='hour', y='rides', data=merged_data, kind='hex')

In [ ]:
merged_data.groupby('hour').rides.mean().plot()